# 🎨 anime-character-generator v1.5 (LoRA Edition)
## Stable Diffusion v1.5 + LoRA Fine-tuning

**バージョン**: 1.5 (LoRA実装版・ブログ範囲内)  
**説明**: Stable Diffusion v1.5 にLoRAによるファインチューニングを適用  
**ガイド**: [Day3-4 ブログ記事](https://shion.blog/)  
**コード**: [train_lora.py](../train_lora.py)

Stable Diffusion v1.5 を **アニメ・特定スタイル専特化** に適用させるため、LoRA を使用したファインチューニングを Google Colab (T4 GPU) で実行します。

このノートブックはブログ記事 Day3-4 の後半部「LoRA ファインチューニング実装」に対応しています。

## 📌 このノートブックについて

**ブログ記事との対応:**
このノートブックは Day3-4 ブログ記事の「Phase 3: 実装から本番化へ」で紹介された `train_lora.py` を、実際に **Google Colab で実行するための実践ガイド**です。

**実装範囲:**
- ✅ Stable Diffusion v1.5 にLoRA の追加学習
- ✅ Google Colab T4 GPU での実行（無料枠対応）
- ✅ チェックポイント保存機能（セッション切断対応）
- ✅ トレーニングログ保存

**学習時間目安（Colab T4）:**
- 1 epoch ≈ 30-40分
- 10 epoch ≈ 5-7時間
- 20 epoch ≈ 10-12時間（推奨は分割実行）

**次のステップ:**
このノートブック完了後、生成されたLoRA重みは Google Drive に保存され、ローカルの `character_generator_v1_lora.py` で利用可能です。

---

## Step 1: 環境確認・GPU セットアップ

In [ ]:
# GPU の確認
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

## Step 2: 依存パッケージのインストール

In [ ]:
# 必要なパッケージをインストール（peft不要 - 純粋PyTorch LoRAを使用）
!pip install -q diffusers transformers pillow torch tqdm safetensors

## Step 3: Google Drive にマウント

In [ ]:
from google.colab import drive

# Google Drive をマウント
drive.mount('/content/drive')
print("✅ Google Drive mounted")

## Step 4: データセットの準備

**事前準備 (このノートブック実行前):**
1. ローカルで `python scripts/download_danbooru.py --limit 60` を実行
2. 生成された `training_data/` フォルダを Google Drive にアップロード
3. 以下のセルを実行

In [ ]:
import os
from pathlib import Path

# Google Drive から training_data をコピー
drive_path = Path("/content/drive/MyDrive/training_data")
local_path = Path("/content/training_data")

if drive_path.exists():
    print(f"✅ Found: {drive_path}")
    # シンボリックリンク作成 (速度向上)
    if not local_path.exists():
        os.symlink(drive_path, local_path)
        print(f"✅ Symlink created: {local_path}")
    
    # データセット統計
    import json
    metadata_file = drive_path / "metadata.json"
    if metadata_file.exists():
        with open(metadata_file) as f:
            metadata = json.load(f)
        print(f"📊 Dataset: {len(metadata.get('training_data', []))} images")
else:
    print(f"❌ Not found: {drive_path}")
    print("\nPlease upload training_data/ to Google Drive")

## Step 5: train_lora.py のダウンロード

GitHub から最新の train_lora.py を取得

In [ ]:
import sys
import urllib.request
from pathlib import Path

print("=" * 70)
print("📥 Step 5: train_lora.py をダウンロード")
print("=" * 70)

train_script = Path("/content/train_lora.py")

# キャッシュクリア（再実行時に古いバージョンが残らないよう）
if "train_lora" in sys.modules:
    del sys.modules["train_lora"]

# GitHub から直接ダウンロード
url = "https://raw.githubusercontent.com/Shion1124/anime-character-generator/master/train_lora.py"

try:
    print(f"📥 ダウンロード中: {url[:60]}...")
    urllib.request.urlretrieve(url, train_script)
    print(f"✅ ダウンロード完了: {train_script}")
    print(f"📦 ファイルサイズ: {train_script.stat().st_size / 1024:.1f} KB")
except Exception as e:
    print(f"⚠️  ダウンロード失敗: {e}")
    print(f"\n🔄 代替方法: git clone を試行...")
    import subprocess, shutil
    try:
        subprocess.run([
            "git", "clone", "--depth", "1",
            "https://github.com/Shion1124/anime-character-generator.git",
            "/tmp/anime-gen"
        ], timeout=30)
        shutil.copy("/tmp/anime-gen/train_lora.py", train_script)
        print(f"✅ git clone からコピー完了")
    except Exception as e2:
        print(f"❌ 失敗: {e2}")
        raise

sys.path.insert(0, "/content")
print(f"✅ Python パス更新完了")

### Step 5.5: (ダウンロード失敗時のみ実行)

上の Step 5 でダウンロードが失敗した場合のみ、以下に進んでください。

In [ ]:
# Step 5.5: もし Step 5 でダウンロード失敗した場合
import subprocess
import shutil
from pathlib import Path

train_script = Path("/content/train_lora.py")

if train_script.exists() and train_script.stat().st_size > 10000:
    print(f"✅ train_lora.py は既に存在します: {train_script.stat().st_size / 1024:.1f} KB")
else:
    print("📥 git clone を試行中...")
    try:
        subprocess.run([
            "git", "clone", "--depth", "1",
            "https://github.com/Shion1124/anime-character-generator.git",
            "/tmp/anime-gen"
        ], timeout=60, capture_output=True)
        shutil.copy("/tmp/anime-gen/train_lora.py", train_script)
        print(f"✅ 取得完了: {train_script.stat().st_size / 1024:.1f} KB")
    except Exception as e:
        print(f"❌ 失敗: {e}")
        print("🔧 手動対応: Google Drive から train_lora.py をアップロードしてください")

### Step 5.6: (それでもダウンロード失敗時のみ実行)

Step 5 と 5.5 での取得に失敗した場合のみ実行します。

In [ ]:
# Step 5.6: 最終手段 - urllib でダウンロード
import urllib.request
import ssl
from pathlib import Path

train_script = Path("/content/train_lora.py")

if train_script.exists() and train_script.stat().st_size > 10000:
    print(f"✅ train_lora.py は既に存在します")
else:
    print("📥 urllib でダウンロード試行...")
    
    ssl_context = ssl.create_default_context()
    ssl_context.check_hostname = False
    ssl_context.verify_mode = ssl.CERT_NONE
    
    url = "https://raw.githubusercontent.com/Shion1124/anime-character-generator/master/train_lora.py"
    
    try:
        request = urllib.request.Request(url)
        request.add_header('User-Agent', 'Mozilla/5.0')
        
        with urllib.request.urlopen(request, context=ssl_context, timeout=30) as response:
            content = response.read().decode('utf-8')
        
        with open(train_script, 'w', encoding='utf-8') as f:
            f.write(content)
        
        print(f"✅ ダウンロード成功: {train_script.stat().st_size / 1024:.1f} KB")
    except Exception as e:
        print(f"❌ すべての取得方法が失敗しました: {e}")
        print("📍 Google Drive から手動でアップロードしてください")

## Step 6: LoRA トレーニング実行

**LoRA (Low-Rank Adaptation) について：**
- Stable Diffusion v1.5 の UNet に疑似的な「アニメ特化」重みを追加学習
- 元のモデルはそのまま（追加パラメータは ~4MB のみ）
- 学習に 20 ステップ × 画像数で推論（3-5秒/画像）

**推奨設定（Colab T4）:**
- エポック: 10-20
- バッチサイズ: 2
- 学習時間: 5-12時間（フルトレーニング）

**重要:** Colab は 12 時間でセッション切断されるため、チェックポイント機能で復帰可能です。

In [ ]:
import torch
import sys
from pathlib import Path

print("=" * 70)
print("🎯 Step 6: LoRA トレーニング")
print("=" * 70)

# 設定パラメータ
EPOCHS = 10
BATCH_SIZE = 2
LEARNING_RATE = 1e-4
LORA_RANK = 8
LORA_ALPHA = 32.0
OUTPUT_DIR = "/content/lora_weights"

print(f"""
⚙️  トレーニング設定:
   エポック数: {EPOCHS}
   バッチサイズ: {BATCH_SIZE}
   学習率: {LEARNING_RATE}
   LoRA ランク: {LORA_RANK}
   LoRA アルファ: {LORA_ALPHA}
   推定時間: 約 {EPOCHS * 30 // 60} 時間 (Colab T4)
""")

# GPU確認
if torch.cuda.is_available():
    total_mem = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"📊 GPU: {torch.cuda.get_device_name(0)} ({total_mem:.1f} GB)")

# データセット確認
training_data_dir = Path("/content/training_data")
total_images = len(list(training_data_dir.rglob("*.png"))) \
             + len(list(training_data_dir.rglob("*.jpg"))) \
             + len(list(training_data_dir.rglob("*.jpeg")))

print(f"📊 トレーニング画像数: {total_images}")

if total_images == 0:
    print("❌ ERROR: トレーニング画像が見つかりません")
    print("   Step 4 でデータセットを準備してください")
    sys.exit(1)

# train_lora.py から LoRATrainer をインポート
sys.path.insert(0, "/content")

try:
    from train_lora import LoRATrainer
    print("✅ train_lora モジュールをインポート\n")

    # トレーナー初期化
    # __init__ のパラメータ: model_name, device, lora_rank, lora_alpha
    print("🚀 LoRA トレーナー初期化中...")
    trainer = LoRATrainer(
        model_name="runwayml/stable-diffusion-v1-5",
        device="cuda" if torch.cuda.is_available() else "cpu",
        lora_rank=LORA_RANK,
        lora_alpha=LORA_ALPHA,
    )

    # トレーニング開始
    # train() のパラメータ: data_dir, output_dir, epochs, batch_size, learning_rate
    print(f"\n🚀 トレーニング開始 ({EPOCHS} epoch × {total_images} images)...\n")

    training_log = trainer.train(
        data_dir="/content/training_data",
        output_dir=OUTPUT_DIR,
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        learning_rate=LEARNING_RATE,
    )

    print(f"\n✅ トレーニング完了！")
    print(f"📁 出力: {OUTPUT_DIR}")

except ImportError as e:
    print(f"❌ インポートエラー: {e}")
    print("   Step 5 を実行して train_lora.py を取得してください")
    raise
except Exception as e:
    print(f"❌ トレーニングエラー: {e}")
    import traceback
    traceback.print_exc()
    raise

print("\n" + "=" * 70)

## Step 7: 学習結果の確認

In [ ]:
import json
from pathlib import Path

output_dir = Path("/content/lora_weights")
print("📁 出力ファイル:")
print()

for file in sorted(output_dir.glob("*")):
    if file.is_file():
        size_mb = file.stat().st_size / (1024 * 1024)
        print(f"  {file.name:40s} {size_mb:>8.2f} MB")

# トレーニングログ確認
log_file = output_dir / "training_log.json"
if log_file.exists():
    with open(log_file) as f:
        log = json.load(f)
    
    print(f"\n✅ トレーニング完了!")
    print(f"   損失: {log['losses'][-1]['loss'] if log.get('losses') else 'N/A':.6f}")
    print(f"   出力ディレクトリ: {output_dir}")
else:
    print(f"\n⚠️  training_log.json が見つかりません")

## Step 7: トレーニング結果の確認

In [ ]:
import os
from pathlib import Path

lora_dir = Path("/content/lora_weights")
print("📁 Output files:")
print()

for file in sorted(lora_dir.glob("*")):
    if file.is_file():
        size_mb = file.stat().st_size / (1024 * 1024)
        print(f"  {file.name:40s} {size_mb:>8.2f} MB")

# メインの LoRA 重みファイル
lora_model = lora_dir / "anime-impressionist-lora.safetensors"
if lora_model.exists():
    size_mb = lora_model.stat().st_size / (1024 * 1024)
    print(f"\n✅ Main LoRA weights: {lora_model.name} ({size_mb:.2f} MB)")

## Step 9: Google Drive に結果を保存

In [ ]:
# トレーニング結果を Google Drive に保存
import shutil
from pathlib import Path

lora_weights_dir = Path("/content/lora_weights")
drive_output_dir = Path("/content/drive/MyDrive/lora_weights")

# 出力ディレクトリ作成
drive_output_dir.mkdir(parents=True, exist_ok=True)

# ファイルをコピー
for file in lora_weights_dir.glob("*"):
    if file.is_file():
        destination = drive_output_dir / file.name
        shutil.copy2(file, destination)
        print(f"✅ 保存: {file.name}")

print(f"\n💾 Google Drive に保存完了: {drive_output_dir}")
print(f"\n実際に使用するには:")
print(f"  1. LoRA重みをダウンロード")
print(f"  2. ローカルで character_generator_v1_lora.py で利用")
print(f"  3. または HuggingFace Hub にアップロード")

## Step 10: 推論テスト (オプション)

トレーニング済み LoRA モデルで推論テストを実行

In [ ]:
# LoRA を使用した推論テスト
from diffusers import StableDiffusionPipeline
import torch
from PIL import Image

print("⏳ Loading model...")

# パイプライン作成
pipe = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=torch.float16
)
pipe = pipe.to("cuda")

# LoRA 重みロード
try:
    # PEFT を使用した LoRA ロード
    from peft import PeftModel
    # (LoRA ロード実装)
    print("✅ LoRA weights loaded")
except:
    print("⚠️ LoRA loading requires additional setup")

# テストプロンプト
prompt = "1girl, watercolor painting style, masterpiece, high quality, anime"
negative_prompt = "low quality, nsfw"

print(f"\n🎨 Generating: {prompt}...")

# 画像生成
with torch.no_grad():
    image = pipe(
        prompt=prompt,
        negative_prompt=negative_prompt,
        height=512,
        width=512,
        num_inference_steps=20,
        guidance_scale=7.0
    ).images[0]

# 結果表示
image.save("/content/test_output.png")
print("✅ Image saved: test_output.png")
display(image)

## ✅ 完了！

**次のステップ:**
1. LoRA 重みをダウンロード
2. ローカルで `character_generator.py --use_lora=True` で推論
3. 結果を確認

**参考:**
- [GitHub リポジトリ](https://github.com/Shion1124/anime-character-generator)
- [HuggingFace モデル](https://huggingface.co/runwayml/stable-diffusion-v1-5)

## Step 11: HuggingFace Hub へのアップロード (オプション)

トレーニング済み LoRA アダプタと **モデルカード (README.md)** を HuggingFace Hub で公開します。

---

### Step 11.1: HuggingFace ログイン

HuggingFace にモデルをアップロードするには、**書き込み権限のあるアクセストークン** が必要です。  
以下の手順でトークンを取得してください。

**ステップ1: 設定画面を開く**
1. [HuggingFace](https://huggingface.co/) にログインします
2. 画面右上の自分のアイコンをクリックします
3. メニューの中から **「Settings」** を選択します

**ステップ2: アクセストークンのページへ**
- 左側のサイドメニューにある **「Access Tokens」** をクリックします

**ステップ3: 新しいトークンを作成する**
1. **「+ Create new token」** ボタンをクリックします
2. 以下の2項目を入力・選択します：
   - **Token Name**: 分かりやすい名前（例: `anime-lora-upload`）
   - **Token type**: 必ず **「Write」** を選択（アップロードには書き込み権限が必要）
3. **「Create token」** ボタンを押して完了

**ステップ4: トークンをコピーして保存する**
- トークン横の **コピーアイコン** をクリックしてコピーします
- メモ帳などに貼り付けて安全に保管してください

> ⚠️ **重要な注意点**  
> トークンは「パスワード」と同じです。GitHub などに貼り付けて公開しないよう、十分注意してください。  
> 次のセルを実行するとプロンプトが表示されるので、そこにトークンを貼り付けてください。

In [ ]:
# -----------------------------
# Step 11.1: HuggingFace ログイン
# -----------------------------
# HuggingFace Hub にモデルをアップロードするためにログインします。
# 実行するとプロンプトが表示されるので、取得した Write トークンを貼り付けてください。

from huggingface_hub import login, HfApi

login()           # Colab will prompt — ここにトークンを貼り付ける
api = HfApi()

print("✅ HuggingFace ログイン完了")

---

### Step 11.2: 学習成果物の確認と README.md（モデルカード）の生成

学習後、`/content/lora_weights/anime-lora-final/` に以下のファイルが保存されています（最低限）：

| ファイル | 説明 |
|---------|------|
| `adapter_config.json` | LoRA 設定（ランク・アルファ値など） |
| `adapter_model.safetensors` | LoRA 重み本体 |
| `training_log.json` | トレーニングログ |

アップロード前に、次のコードで **README.md（モデルカード）** を自動生成します。

---

#### README.md の正しい書き方

HuggingFace では `README.md` = **モデルカード** です。  
「このLoRAは何を学習し、どう使い、何に注意すべきか」を 第三者が再利用できる水準で説明する必要があります。

**必須構成（この順で書くこと）：**

**① YAML メタデータ（必須）**
```yaml
---
base_model: runwayml/stable-diffusion-v1-5
datasets:
- danbooru
language:
- ja
- en
license: openrail
library_name: peft
pipeline_tag: text-to-image
tags:
- lora
- stable-diffusion
- anime
---
```
YAML front matter がないと HuggingFace で「壊れたモデルカード」扱いになります。

**② モデル概要（What）** — 「LoRA アダプタのみ」であることを明記  
**③ 学習目的・スタイル（Why）** — どのアニメスタイルを学習したか  
**④ 学習設定（How）** — ランク・アルファ・エポック数・学習率  
**⑤ 使用方法（How to use）** — コピペで動くコード例  
**⑥ データセット・ライセンス（必須）** — Danbooru / OpenRAIL 準拠

> ⚠️ **課題**  
> モデルタイトル（`title_line`）は必ず自分で記入してください。  
> データセットを変更した場合は Sources & Terms 欄も更新してください。

In [ ]:
# -----------------------------
# Step 11.2: README.md（モデルカード）を生成
# -----------------------------
# 学習完了後に実行し、HuggingFace の README.md（モデルカード）を生成します。
# 学習ハイパーパラメータはコードの変数から自動同期されます。

import os
import json
from pathlib import Path

OUT_LORA_DIR = "/content/lora_weights/anime-lora-final"
os.makedirs(OUT_LORA_DIR, exist_ok=True)

# ------------------------------------------------------------------
# Step 6 変数の読み込み（未定義の場合はデフォルト値を使用）
# ------------------------------------------------------------------
def _s(x, default=""):
    try:
        v = str(x)
        return v if v.strip() else default
    except Exception:
        return default

def _fmt_lr(x) -> str:
    try:
        return f"{float(x):.0e}"
    except Exception:
        return _s(x, "")

try:
    _lora_rank  = int(LORA_RANK)
    _lora_alpha = float(LORA_ALPHA)
    _epochs     = int(EPOCHS)
    _lr_str     = _fmt_lr(LEARNING_RATE)
    _batch_size = int(BATCH_SIZE)
except NameError:
    print("⚠️  Step 6 の変数が見つかりません。デフォルト値を使用します")
    _lora_rank  = 8
    _lora_alpha = 32.0
    _epochs     = 10
    _lr_str     = "1e-4"
    _batch_size = 2

# トレーニングログから最終損失を取得
_final_loss = "N/A"
log_file = Path("/content/lora_weights/training_log.json")
if log_file.exists():
    with open(log_file) as f:
        log = json.load(f)
    if log.get("losses"):
        _final_loss = f"{log['losses'][-1]['loss']:.6f}"
        _epochs = len(log["losses"])

# ------------------------------------------------------------------
# README の設定
# ⚠️ MODEL_TITLE は必ず自分で記入すること
# ------------------------------------------------------------------
BASE_MODEL_ID = "runwayml/stable-diffusion-v1-5"
HF_REPO_ID    = "YOUR_USERNAME/anime-character-lora"  # ← 自分のリポジトリIDに変更
PRIVATE       = False                                  # True でプライベート公開

# README.md タイトル（必ず自分で記入してください）
title_line    = "anime-character-lora"  # 例: my-anime-watercolor-lora

# ------------------------------------------------------------------
# README.md 本文の生成
# ------------------------------------------------------------------
readme_md = f"""---
base_model: {BASE_MODEL_ID}
datasets:
- danbooru
language:
- ja
- en
license: openrail
library_name: peft
pipeline_tag: text-to-image
tags:
- lora
- stable-diffusion
- anime
- character-generation
- fine-tuned
---

# {title_line}

This repository provides a **LoRA adapter** fine-tuned from  
**{BASE_MODEL_ID}** using **PyTorch LoRA (PEFT)**.

This repository contains **LoRA adapter weights only**.  
The base model must be loaded separately.

## Training Objective

This adapter is trained to improve **anime character generation quality**  
across 5 artistic styles (impressionist, soft-focus, oil painting, sketch, pastel).

The model learns style-specific features from Danbooru anime images,  
applied to the UNet attention layers (`to_k`, `to_v`, `to_q`, `to_out.0`).

## Example Output

![Example generated image](examples/test_output.png)

## Training Configuration

- Base model: `{BASE_MODEL_ID}`
- Method: LoRA (PEFT)
- Target modules: `to_k`, `to_v`, `to_q`, `to_out.0` (Attention Linear layers only)
- LoRA rank: {_lora_rank}
- LoRA alpha: {_lora_alpha}
- Learning rate: {_lr_str}
- Batch size: {_batch_size}
- Epochs: {_epochs}
- Final loss: {_final_loss}
- GPU: Colab T4 (16 GB VRAM)

## Dataset

Danbooru anime images collected and classified into 5 styles:

| Style | Images |
|-------|--------|
| impressionist_style | ~60 |
| soft_focus_landscape | ~60 |
| oil_painting_aesthetic | ~60 |
| sketch_aesthetic | ~60 |
| pastel_softness | ~60 |

Total: ~300 images

## Usage

```python
import torch
from diffusers import StableDiffusionPipeline
from peft import PeftModel

base = "{BASE_MODEL_ID}"
adapter = "{HF_REPO_ID}"

pipe = StableDiffusionPipeline.from_pretrained(base, torch_dtype=torch.float16)
pipe.unet = PeftModel.from_pretrained(pipe.unet, adapter, adapter_name="anime_lora")
pipe = pipe.to("cuda")

image = pipe(
    prompt="1girl, anime character, watercolor style, masterpiece, high quality",
    negative_prompt="low quality, blurry, distorted, nsfw",
    num_inference_steps=20,
    guidance_scale=7.5,
    height=512,
    width=512,
    generator=torch.Generator(device="cuda").manual_seed(42)
).images[0]
image.save("output.png")
```

## Recommended Prompts

**Anime character:**
```
1girl, anime character, detailed beautiful face, long hair,
watercolor painting style, soft colors, bokeh background,
masterpiece, best quality, high quality, intricate details
```

**Negative prompt:**
```
low quality, worst quality, blurry, distorted, watermark,
error, nsfw, extra limbs, missing limbs, ugly, bad anatomy
```

| Parameter | Value |
|-----------|-------|
| `num_inference_steps` | 20 |
| `guidance_scale` | 7.5 |
| `height / width` | 512 × 512 |

## Sources & Terms (IMPORTANT)

Training data: Danbooru (https://danbooru.donmai.us/)

Dataset License: CC0 (Public Domain). Images sourced from Danbooru under CC0 terms.  
Compliance: Users must comply with the base model's original license terms (OpenRAIL-M).

## License

| Component | License |
|-----------|---------|
| Stable Diffusion v1.5 | OpenRAIL-M |
| LoRA adapter (this repo) | Apache 2.0 |
| Training data (Danbooru) | CC0 |

## References

1. Ho et al. (2020) - *Denoising Diffusion Probabilistic Models* - [arXiv:2006.11239](https://arxiv.org/abs/2006.11239)
2. Rombach et al. (2022) - *High-Resolution Image Synthesis with Latent Diffusion Models* - [arXiv:2112.10752](https://arxiv.org/abs/2112.10752)
3. Hu et al. (2021) - *LoRA: Low-Rank Adaptation of Large Language Models* - [arXiv:2106.09685](https://arxiv.org/abs/2106.09685)
"""

# ------------------------------------------------------------------
# README.md の書き込み
# ------------------------------------------------------------------
readme_path = os.path.join(OUT_LORA_DIR, "README.md")
with open(readme_path, "w", encoding="utf-8") as f:
    f.write(readme_md)

# ------------------------------------------------------------------
# 動作確認
# ------------------------------------------------------------------
assert os.path.exists(readme_path), "README.md was not written."
assert readme_md.lstrip().startswith("---\n"), (
    "README.md must start with YAML front matter (---)."
)
assert readme_md.count("\n---\n") >= 1, (
    "YAML front matter must be closed with ---."
)

print(f"✅ README.md を生成しました: {readme_path}")
print(f"   サイズ: {Path(readme_path).stat().st_size / 1024:.1f} KB")
print()
print("📄 プレビュー（先頭30行）:")
for i, line in enumerate(readme_md.splitlines()[:30], start=1):
    print(f"{i:02d}: {line}")

---

### Step 11.3: LoRA アダプタを HuggingFace Hub へアップロード

README.md の生成が完了したら、以下のコードでアップロードします。

**アップロード前の確認事項：**
- Step 11.2 の README.md 生成が完了していること
- `adapter_config.json` と `adapter_model.safetensors`（または `.bin`）が存在すること
- `HF_REPO_ID` が正しく設定されていること（Step 11.2 のコード冒頭で変更）

**アップロードされるファイル（ホワイトリスト）：**

| ファイル | 必須 |
|---------|------|
| `README.md` | ✅ 必須 |
| `adapter_config.json` | ✅ 必須 |
| `adapter_model.safetensors` / `.bin` | ✅ 必須 |
| `tokenizer.*` / `special_tokens_map.json` | 任意 |
| `*.json` | 任意 |

> ⚠️ 不要な一時ファイルはホワイトリストで除外されます。  
> 必須ファイルが欠けている場合はエラーで停止します。

In [ ]:
# ============================================================
# Step 11.3: LoRA アダプタを HuggingFace Hub へアップロード
# ============================================================
# README.md が存在しない場合はエラーで停止します。
# 不要な一時ファイルはホワイトリストで除外し、ステージング領域経由でアップロードします。

import fnmatch
import shutil
from pathlib import Path
from huggingface_hub import HfApi

# HuggingFace API インスタンス（Step 11.1 の login() 済みであること）
api = HfApi()

# アップロード設定（Step 11.2 の HF_REPO_ID / PRIVATE を引き継ぐ）
try:
    _repo_id = HF_REPO_ID
    _private = PRIVATE
except NameError:
    _repo_id = "YOUR_USERNAME/anime-character-lora"
    _private = False
    print("⚠️  HF_REPO_ID 未定義。Step 11.2 のコードを先に実行してください")

LORA_SAVE_DIR = Path(OUT_LORA_DIR)

# -----------------------------
# 3.1) 必須ファイルの存在確認
# -----------------------------
required_files = {
    "adapter_config.json",  # LoRA 設定ファイル
    "README.md",            # モデルカード（Step 11.2 で生成）
}

present = {p.name for p in LORA_SAVE_DIR.iterdir() if p.is_file()}
missing = [f for f in required_files if f not in present]

# adapter_model.safetensors または .bin の存在確認
if not any(f.startswith("adapter_model.") for f in present):
    missing.append("adapter_model.(safetensors|bin)")

if missing:
    raise RuntimeError(
        "アップロードを中止しました。\n"
        "以下の必須ファイルが見つかりません:\n"
        + "\n".join(f"  - {m}" for m in missing) +
        "\n\n"
        "対処法:\n"
        "  - README.md → Step 11.2 を実行してください\n"
        "  - adapter_config.json / adapter_model.* → Step 6 (トレーニング) を完了してください"
    )

print("✅ 必須ファイルの確認が完了しました")
print(f"   保存先: {LORA_SAVE_DIR}")
for f in sorted(present):
    size_kb = (LORA_SAVE_DIR / f).stat().st_size / 1024
    print(f"   {f:45s} {size_kb:>8.1f} KB")

# -----------------------------
# 3.2) アップロード対象の選別（ホワイトリスト）
# 3.2.1) LoRA 重みファイル
# 3.2.2) サンプル画像（test_output.png）
# 3.2.3) モデルカード（README.md）
# 3.2.4) 設定ファイル（adapter_config.json）
# -----------------------------
# 不要な一時ファイルを除外し、許可するファイル形式のみをアップロード
ALLOW_PATTERNS = [
    "README.md",
    "adapter_config.json",
    "adapter_model.*",
    "tokenizer.*",
    "special_tokens_map.json",
    "*.json",
    "examples/*",  # ← サンプル画像フォルダ
]

def is_allowed(name: str) -> bool:
    return any(fnmatch.fnmatch(name, pat) for pat in ALLOW_PATTERNS)

# ステージング領域（一時フォルダ）を作成
STAGE_DIR = Path("/content/hf_upload_stage")
if STAGE_DIR.exists():
    shutil.rmtree(STAGE_DIR)
STAGE_DIR.mkdir(parents=True)

# 許可されたファイルのみをコピー
for p in LORA_SAVE_DIR.iterdir():
    if p.is_file() and is_allowed(p.name):
        (STAGE_DIR / p.name).write_bytes(p.read_bytes())

# ========== 追加: テスト画像を examples フォルダに含める ==========
test_image_path = Path("/content/test_output.png")
if test_image_path.exists():
    examples_dir = STAGE_DIR / "examples"
    examples_dir.mkdir(parents=True, exist_ok=True)
    (examples_dir / "test_output.png").write_bytes(test_image_path.read_bytes())
    print(f"✅ サンプル画像を examples フォルダに追加")
else:
    print(f"⚠️  テスト画像が見つかりません: {test_image_path}")
    print(f"   (Step 10 の推論テストを実行してください)")

staged_files = [p.name for p in sorted(STAGE_DIR.rglob("*")) if p.is_file()]
print(f"\n📦 アップロード対象ファイル ({len(staged_files)}個):")
for f in staged_files:
    size_kb = (STAGE_DIR / f).stat().st_size / 1024
    print(f"   {f:45s} {size_kb:>8.1f} KB")

# -----------------------------
# 3.3) リポジトリ作成とアップロード
# -----------------------------
print(f"\n🚀 リポジトリを作成/確認中: {_repo_id}")
api.create_repo(
    repo_id=_repo_id,
    repo_type="model",
    exist_ok=True,
    private=_private,
)

print(f"📤 アップロード中...")
api.upload_folder(
    folder_path=str(STAGE_DIR),
    repo_id=_repo_id,
    repo_type="model",
    commit_message=f"Upload LoRA adapter (rank={_lora_rank}, alpha={_lora_alpha}, epochs={_epochs})",
)

# ステージング領域を削除
shutil.rmtree(STAGE_DIR)

# -----------------------------
# 完了メッセージ
# -----------------------------
print()
print("=" * 60)
print("🎉 HuggingFace Hub へのアップロード完了！")
print("=" * 60)
print(f"\n🔗 モデルURL: https://huggingface.co/{_repo_id}")
print(f"\n📖 使用方法:")
print(f"   from diffusers import StableDiffusionPipeline")
print(f"   from peft import PeftModel")
print(f"   import torch")
print(f"   pipe = StableDiffusionPipeline.from_pretrained(")
print(f"       'runwayml/stable-diffusion-v1-5', torch_dtype=torch.float16")
print(f"   )")
print(f"   pipe.unet = PeftModel.from_pretrained(pipe.unet, '{_repo_id}')")
print(f"   pipe = pipe.to('cuda')")